In [2]:
import anthropic
import os

path = '/work/bioinformatics/s229618/api_keys/claude_key.txt'

with open(path, 'r') as file:
    key = file.read()

client = anthropic.Anthropic(
    api_key=key,
)
message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=1000,
    temperature=0,
    system="Respond only in pirate speak.",
    messages=[
        {"role": "user", "content": "Hi Claude."}
    ]
)
print(message.content)

[TextBlock(text="Ahoy matey! 'Tis a fine day to be sailin' the high seas o' conversation, isn't it? What winds be blowin' ye way on this voyage?", type='text')]


In [3]:
transcript_id = '10_0991_331330'
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer

def read_transcript_from_id(transcript_id):

    path_to_data_folder = '/archive/shared/sim_center/shared/ameer/'
    # lookinto this dictionary to find the path
    # can also manually create the path and it would be faster but not by much
    merged_lookup = pd.read_csv(path_to_data_folder + 'grade_lookupv5.csv')
    transcript = ''

    path = merged_lookup[merged_lookup.id == transcript_id].path.iloc[0]
    with open(path, 'r') as file:
        lines = file.readlines()
    for line in lines:
        if line != '\n':
            tok_line = line.split(' ')
            for i in range(len(tok_line)):
                transcript += ' ' + tok_line[i]


    return transcript

In [4]:
transcript = read_transcript_from_id(transcript_id)

In [5]:
def summarize(transcript):
    summary = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        temperature=0,
        system="Please summarize the following text in one paragraph. 100 words. Do not add any information that is not in the text.",
        messages=[
            {"role": "user", "content": transcript}
        ]
    )

    return summary
#print(message.content)

In [8]:
#summary.content[0].text

In [6]:
def diarize(summary, transcript):
    diarization = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        temperature=0,
        system='''The following text represents an audio transcript between a medical student and a patient, with the summary included below.
                The student is the one who will be playing the role of doctor, asking questions about the patient's condition and symptoms.
                Please diarize the following transcript in order to indicate who speaks when, using the format in the following example, with the label "Student" for the medical student and "Patient" for the patient (not their names):

                Student: Hello, how are you today?

                Patient: I am feeling sick. 

                Student: Oh no, how long have you felt sick? 

                Patient: About two days. 

                Separate speaker turns by an extra new line, as above ('\n\n'). The lines do not always have to be alternating labels. There may be consecutive lines from one speaker. Do not use any speaker labels for the two participants other than "Student" and "Patient."

                The transcript may contain some instructions for the student, coming from a third-party speaker. Please label this segment "Instructions: ". 
                If there are no instructions in the transcript, disregard this label. 

                IMPORTANT: Do not remove or add any words to the transcript other than the speaker labels. Also, do not add or remove any punctuation or change any spellings.''',
        messages=[
            {"role": "user", "content": 'summary: ' + summary.content[0].text + '\n\ntranscript: ' + transcript}
        ]
    )

    return diarization
#print(diarization.content)

In [10]:
#print(diarization.content[0].text)

In [7]:
def save_file(diarization, transcript_id):
    path = '/archive/shared/sim_center/shared/annie/claude/'
    with open(path + transcript_id + ".txt", "w") as outfile:
        outfile.write('ID: ' + transcript_id + '\n\n' + diarization.content[0].text)

In [8]:
def claude_diarize(transcript_id):
    print(transcript_id)
    transcript = read_transcript_from_id(transcript_id)
    summary = summarize(transcript)
    diarization = diarize(summary, transcript)
    save_file(diarization, transcript_id)

In [9]:
id_set1 = ['01_0542_298135',
'02_0036_174595',
'03_0028_174553',
'04_0043_174686',
'05_0033_174804',
'06_0079_175106',
'07_0068_174641',
'08_0029_174576',
'09_0029_174582',
'10_0991_331330']

id_set2 = ['01_1080_366142',
           '02_1056_380177',
           '03_1500_380168',
           '04_1512_380182',
           '05_1066_380195',
           '06_1048_365209',
           '07_1111_380134',
           '08_1044_380133',
           '09_1039_380193',
           '10_1005_331402'
]

In [14]:
for id in id_set1:
    claude_diarize(id)

In [10]:
for id in id_set2:
    claude_diarize(id)

01_1080_366142


02_1056_380177
03_1500_380168
04_1512_380182
05_1066_380195
06_1048_365209
07_1111_380134
08_1044_380133
09_1039_380193
10_1005_331402
